<a href="https://colab.research.google.com/github/JannatulJyoti122/Thesis_Undergrade/blob/main/RAQ_QA_ChatHistory_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain chromadb faiss-cpu openai tiktoken langchain_openai langchain-community wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.4/438.4 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load the text file
file_path = "constitution.txt"  # Replace with your actual file path
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,       # max characters per chunk
    chunk_overlap=70  ,    # overlap between chunks to preserve context
    separators=["\n\n", "\n", "।", ".", " "]
)

# Split the text (pass the variable, not a string)
chunks = text_splitter.split_text(text)
print(f"Created {len(chunks)} chunks ")
# Print the chunks
for i, chunk in enumerate(chunks[:10]):
    print(f"Chunk {i+1}:\n{chunk}\n")


Created 324 chunks 
Chunk 1:
প্রস্তাবনা
আমরা, বাংলাদেশের জনগণ , ১৯৭১ সালের ২৬শে মার্চ স্বাধীনতা ঘোষণা করে জাতীয় মুক্তির ঐতিহাসিক সংগ্রামের মাধ্যমে স্বাধীন ও সার্বভৌম গণপ্রজাতন্ত্রী বাংলাদেশ প্রতিষ্ঠা করেছি।

Chunk 2:
আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।

Chunk 3:
আমরা আরও অঙ্গীকার করছি, আমাদের রাষ্ট্রের অন্যতম লক্ষ্য হবে গণতান্ত্রিক পদ্ধতিতে একটি শোষণমুক্ত সমাজতান্ত্রিক সমাজ গঠন, যেখানে আইনের শাসন, মৌলিক মানবাধিকার এবং রাজনৈতিক, অর্থনৈতিক ও সামাজিক সমতা, স্বাধীনতা ও ন্যায্যতা সব নাগরিকের জন্য নিশ্চিত করা হবে।

Chunk 4:
আমরা জোর দিয়ে ঘোষণা করছি, যেন আমরা স্বাধীন জাতি হিসেবে উন্নতি করতে পারি এবং মানবজাতির অগ্রগতিশীল আশা ও আকাঙ্ক্ষার সঙ্গে সঙ্গতি রেখে আন্তর্জাতিক শান্তি ও সহযোগিতায় সম্পূর্ণ ভূমিকা রাখতে পারি, সে উদ্দেশ্যে এই সংবিধানের শ্রেষ্ঠত্ব বজায় রাখা এবং একে রক্ষা, সমর্থন

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses, util
from torch.utils.data import DataLoader
from sentence_transformers.datasets import ParallelSentencesDataset
import logging
import os
import numpy as np


class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        """Initialize the model once"""
        self.model = SentenceTransformer(model_name)
    def encode(self, sentences):
        """Encode a list of sentences"""
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))




In [ ]:
from langchain.docstore.document import Document
import chromadb
from langchain.embeddings import SentenceTransformerEmbeddings
 # Assuming this is your custom class

# 1. Initialize the Bangla Sentence Transformer
model = BanglaSentenceTransformerSmall()

# 2. Encode the chunks
embeddings_dict = model.encode(chunks)

# 3. Prepare documents with the chunk text
documents = [Document(page_content=chunk) for chunk in embeddings_dict.keys()]

# 4. Set up ChromaDB client
client = chromadb.Client()

# 5. Try to get the collection if it already exists
collection_name = "bangla_embeddings"
try:
    collection = client.get_collection(collection_name)
except Exception as e:
    # If collection doesn't exist, create it
    collection = client.create_collection(collection_name)

# 6. Add the sentence embeddings to ChromaDB collection
for idx, (sentence, embedding) in enumerate(embeddings_dict.items()):
    # Ensure idx is defined within the loop, no error in 'for' loop
    collection.add(
        ids=[f"doc_{idx}"],             # Unique ID for each document
        documents=[sentence],           # The sentence content
        metadatas=[{"source": "constitution"}],  # Added a key-value pair for metadata
        embeddings=[embedding]          # The corresponding embedding
    )
#client.persist()

print(f"Stored {len(chunks)} chunks in Chroma vector store.")

#print(f"Stored {len(chunks)} chunks in Chroma vector store.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Stored 324 chunks in Chroma vector store.


In [ ]:
from chromadb.config import Settings
client = chromadb.PersistentClient(path="my_chroma_db")


In [ ]:
query = "কখন রাষ্ট্রদ্রোহিতা হবে?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: ৭ক। সংবিধান বাতিল, স্থগিতকরণ, রাষ্ট্রদ্রোহিতা ইত্যাদি অপরাধ :
১। কেউ যদি শক্তি ব্যবহার করে, বা অন্য কোনো বেআইনি উপায়ে—
এই সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা সেটা করার চেষ্টা করে বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে—
তাহলে সেটি রাষ্ট্রদ্রোহিতা হিসেবে গণ্য হবে। সেই ব্যক্তি রাষ্ট্রদ্রোহের অপরাধে অপরাধী বলে ধরা হবে।
Similarity Score: 0.3422

Document: দ্বিতীয় ভাগ, রাষ্ট্র পরিচালনার মূলনীতি
(৮ )(১) জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।
Similarity Score: 0.3931

Document: ধর্ম, প্রভৃতি কারণে বৈষম্য
২৮।(১) কোনো নাগরিকের প্রতি কেবল ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে রাষ্ট্র বৈষম্য করবে না।
(২) রাষ্ট্র ও সমাজের সব স্তরে নারী ও পুরুষের সমান অধিকার নিশ্চিত করবে।
(৩) ধর্ম, গোষ্ঠী, বর্ণ, লিঙ্গ বা জন্মস্থানের কারণে কোনো নাগরিককে বিনোদন বা বিশ্রামের স্থানে প্রবেশ বা শিক্ষা প্রতিষ্ঠানে ভর্তিতে কোনো বাধা, শর্ত বা অযোগ্যতা দেও

In [ ]:
query = "সংবিধানের মূলনীতি?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।
Similarity Score: 0.3194

Document: দ্বিতীয় ভাগ, রাষ্ট্র পরিচালনার মূলনীতি
(৮ )(১) জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।
Similarity Score: 0.3262

Document: তৃতীয় ভাগ, মৌলিক অধিকার
Similarity Score: 0.3424



In [ ]:
query = "সংবিধানে মানবাধিকার নিয়ে কী বলা আছে?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: (৩) সংবিধানে যা বলা হয়েছে, তবুও গণহত্যা, মানবতাবিরোধী অপরাধ, যুদ্ধাপরাধ এবং আন্তর্জাতিক আইনের অধীন অন্যান্য অপরাধের জন্য যেকোনো সশস্ত্র বাহিনী, প্রতিরক্ষা বাহিনী, সহায়ক বাহিনী বা অন্য ব্যক্তি, গোষ্ঠী বা সংগঠন, কিংবা যুদ্ধবন্দীকে আটক, বিচার বা দণ্ডিত করার জন্য কোনো আইন সংবিধানের সঙ্গে অসামঞ্জস্য হলেও তা বাতিল বা অবৈধ বিবেচিত হবে না এবং কখনো বাতিলও হবে না।
Similarity Score: 0.3427

Document: জীবন ও ব্যক্তি-স্বাধীনতার অধিকার রক্ষণ
৩২। কোনো ব্যক্তিকে আইন অনুযায়ী ছাড়া তার জীবন বা ব্যক্তিস্বাধীনতা থেকে বঞ্চিত করা যাবে না।
Similarity Score: 0.3496

Document: আমরা অঙ্গীকার করছি, সেই সব মহান আদর্শ—যেগুলো আমাদের সাহসী জনগণকে জাতীয় মুক্তি সংগ্রামে আত্মনিয়োগ করতে এবং অনেক বীর শহীদকে জীবন উৎসর্গ করতে অনুপ্রাণিত করেছিল—যেমন: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা—এই আদর্শগুলোই হবে এই সংবিধানের মূলনীতি।
Similarity Score: 0.3539



In [ ]:
query = "সংবিধানে বাংলাদেশের জাতীয় প্রতীক নিয়ে কী বলা আছে?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: ৪। জাতীয় সঙ্গীত, পতাকা ও প্রতীক
(১) বাংলাদেশের জাতীয় সঙ্গীত হলো —
“আমার সোনার বাংলা” গানটির প্রথম ১০টি লাইন।
(২) জাতীয় পতাকার বর্ণনা:
সবুজ রঙের পটভূমিতে একটি লাল বৃত্ত (রক্তরঙের পূর্ণ গোল)।
(৩) জাতীয় প্রতীকের বর্ণনা:
পানির উপর ভাসছে জাতীয় ফুল শাপলা।চারপাশে ঘিরে আছে ধানের শীষ।উপরে আছে তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।
Similarity Score: 0.2906

Document: প্রবর্তন, উল্লেখ ও নির্ভরযোগ্য পাঠ (ধারা ১৫৩)
(১) এই সংবিধানকে "গণপ্রজাতন্ত্রী বাংলাদেশের সংবিধান" নামে উল্লেখ করা হবে এবং এটি ১৯৭২ সালের ১৬ ডিসেম্বর থেকে বলবৎ থাকবে, যাকে সংবিধানে "সংবিধান-প্রবর্তন" বলা হয়েছে।
Similarity Score: 0.3379

Document: "প্রজাতন্ত্র": গণপ্রজাতন্ত্রী বাংলাদেশ।
"প্রজাতন্ত্রের কর্ম": বাংলাদেশ সরকারের সামরিক বা অসামরিক যে কোনো চাকুরি বা কর্ম, বা আইনের দ্বারা প্রজাতন্ত্রের কর্ম বলিয়া ঘোষিত অন্য কোনো কাজ।
"প্রধান নির্বাচন কমিশনার": সংবিধানের ১১৮ অনুচ্ছেদ অনুযায়ী নিযুক্ত ব্যক্তি।
"প্রধান বিচারপতি": বাংলাদেশের প্রধান বিচারপতি।
Similarity Score: 0.3425



In [ ]:
query = "বাংলাদেশের জাতীয় সঙ্গীত কী?"
query_embedding = model.encode([query])[query]  # Get the embedding

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["documents", "distances"]  # Important: request distances
)

# Display each result with its similarity score
for doc, score in zip(results['documents'][0], results['distances'][0]):
    print(f"Document: {doc}")
    print(f"Similarity Score: {score:.4f}\n")

Document: ৪। জাতীয় সঙ্গীত, পতাকা ও প্রতীক
(১) বাংলাদেশের জাতীয় সঙ্গীত হলো —
“আমার সোনার বাংলা” গানটির প্রথম ১০টি লাইন।
(২) জাতীয় পতাকার বর্ণনা:
সবুজ রঙের পটভূমিতে একটি লাল বৃত্ত (রক্তরঙের পূর্ণ গোল)।
(৩) জাতীয় প্রতীকের বর্ণনা:
পানির উপর ভাসছে জাতীয় ফুল শাপলা।চারপাশে ঘিরে আছে ধানের শীষ।উপরে আছে তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।
Similarity Score: 0.2818

Document: ২(ক)। রাষ্ট্রধর্ম
বাংলাদেশের রাষ্ট্রধর্ম হলো ইসলাম।তবে হিন্দু, বৌদ্ধ, খ্রিস্টানসহ সব ধর্মের মানুষ যাতে সমান মর্যাদা ও অধিকার নিয়ে নিজের ধর্ম পালন করতে পারে, রাষ্ট্র তা নিশ্চিত করবে।

৩। রাষ্ট্রভাষা
বাংলাদেশের রাষ্ট্রভাষা হবে বাংলা।
Similarity Score: 0.4090

Document: জাতীয়তাবাদ
৯। বাঙালী জাতি হলো এক ভাষা আর সংস্কৃতির মানুষের দল যারা ঐক্যবদ্ধ হয়ে দৃঢ় সংকল্প নিয়ে সংগ্রাম করেছে। তাদের সেই সংগ্রামের ফলেই বাংলাদেশ স্বাধীনতা ও সার্বভৌমত্ব পায়। এই একতা আর সংহতি হলো বাঙালী জাতীয়তাবাদের মূল ভিত্তি।
Similarity Score: 0.4312



In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import SentenceTransformer

In [ ]:
# Step 1: Initialize models
#embedding_model = SentenceTransformer('sagorsarker/bangla-bert-base')  # Better for Bengali
import os
os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(
    openai_api_key="",
    model_name="gpt-3.5-turbo-0125",  # Use a model with better multilingual support
    temperature=0
)
parser = StrOutputParser()

In [ ]:
# Step 2: Initialize Chat History
chat_history = ChatMessageHistory()
def format_chat_history(history, max_messages=10):
    """Convert chat history into a formatted string."""
    messages = history.messages[-max_messages:]
    formatted = ""
    for msg in messages:
        if isinstance(msg, HumanMessage):
            formatted += f"User: {msg.content}\n"
        elif isinstance(msg, AIMessage):
            formatted += f"Assistant: {msg.content}\n"
    return formatted

In [ ]:
# Step 3: Define a function to format documents
def format_docs(results):
    """Format vector store results into a string."""
    #print("Raw results structure:", results)  # Debug full results
    documents = results.get('documents', [[]])[0]  # Get first list of documents
    #print("Extracted documents:", documents)  # Debug extracted documents
    if not documents:
        print("Warning: No documents retrieved!")
        return ""
    formatted = "\n".join([str(doc) for doc in documents if doc])  # Ensure no empty strings
    #print("Formatted context:", formatted[:200], "...")  # Debug formatted output
    if not formatted.strip():
        print("Warning: Formatted context is empty after processing!")
    return formatted  # Limit context length

In [ ]:
# Step 4: Define the vector store query function
def query_vector_store(question):
    embedding = model.encode([question])[question]
    results = collection.query(query_embeddings=[embedding], n_results=3)
    return results

In [ ]:
# Step 5: Wrap components in LangChain Runnables
query_chain = RunnableLambda(query_vector_store)
context_chain = query_chain | RunnableLambda(format_docs)

In [ ]:
# Step 6: Update the Prompt Template
prompt = PromptTemplate(
    template="""
      আপনি বাংলাদেশের সংবিধানের একটি সহায়ক সহকারী।
      শুধুমাত্র নিচের প্রাসঙ্গিক অংশের ভিত্তিতে প্রশ্নের সঠিক এবং সংক্ষিপ্ত উত্তর দিন।
      চ্যাট ইতিহাস:
      {history}
      প্রাসঙ্গিক অংশ:
      {context}
      প্রশ্ন:
      {question}
      উত্তর শুধুমাত্র প্রাসঙ্গিক অংশ থেকে নিন। কোনো বাহ্যিক তথ্য, অনুমান, বা সাধারণ জ্ঞান ব্যবহার করবেন না।
      যদি উত্তর প্রাসঙ্গিক অংশে না থাকে, তবে বলুন: "উত্তর প্রাসঙ্গিক অংশে পাওয়া যায়নি।"
      উত্তর বাংলায়, সংক্ষিপ্ত এবং সঠিক হতে হবে।
    """,
    input_variables=['history', 'context', 'question']
)

In [ ]:
# Step 7: Update the parallel chain to include history
def get_history(_):
    return format_chat_history(chat_history)

parallel_chain = RunnableParallel({
    'history': RunnableLambda(get_history),
    'context': context_chain,
    'question': RunnablePassthrough()
})

In [ ]:

# Step 8: Define the main chain with debugging
def debug_invoke(input):
    #print("Prompt input:", input)
    return prompt.invoke(input)

main_chain = parallel_chain | debug_invoke | llm | parser

In [ ]:
# Step 9: Function to invoke the chain and update history
def ask_question(question):
    try:
        chat_history.add_message(HumanMessage(content=question))
        result = main_chain.invoke(question)
        #print("Raw LLM output:", result)
        chat_history.add_message(AIMessage(content=result))
        return result
    except Exception as e:
        print(f"Error during generation: {e}")
        return None

In [ ]:
# Example usage
question = "কখন স্বাধীনতার ঘোষণা দেওয়া হয়?"
response = ask_question(question)
print("Response:", response)


Response: স্বাধীনতার ঘোষণা ১৯৭১ সালের ২৬শে মার্চে দেওয়া হয়।


In [ ]:
# Example usage
question = "সংবিধানে মানবাধিকার নিয়ে কী বলা আছে?"
response = ask_question(question)
print("Response:", response)

Response: মানবাধিকার সংবিধানে অনেক গুরুত্বপূর্ণ বিষয় নিয়ে বলা হয়েছে। মানবাধিকার নিশ্চিত করা, মানুষের মর্যাদা ও মূল্য সম্মান করা, এবং সব মানুষের স্বাধীনতা ও অধিকার নিশ্চিত করা হয়েছে।


In [ ]:
# Example usage
question = "কখন রাষ্ট্রদ্রোহিতা হবে?"
response = ask_question(question)
print("Response:", response)

Response: উত্তর প্রাসঙ্গিক অংশ থেকে রাষ্ট্রদ্রোহিতা হবে যদি কেউ শক্তি ব্যবহার করে বা অন্য কোনো বেআইনি উপায়ে সংবিধান বা এর কোনো অংশ বাতিল বা স্থগিত করে, বা ষড়যন্ত্র করে, বা জনগণের মধ্যে সংবিধানের প্রতি বিশ্বাস কমানোর চেষ্টা করে। এই অপরাধে সেটি রাষ্ট্রদ্রোহিতা হিসেবে গণ্য হবে এবং তার বিরুদ্ধে আইনানুযায়ী ব্যবস্থা গ্রহণ করা হবে।


In [ ]:
# Example usage
question = "বাংলাদেশের জাতীয় প্রতীক "
response = ask_question(question)
print("Response:", response)

Response: জাতীয় প্রতীকের বর্ণনা: পানির উপর ভাসছে জাতীয় ফুল শাপলা। চারপাশে ঘিরে আছে ধানের শীষ। উপরে আছে তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা)।


In [ ]:
# Example usage
follow_up_question = "এর চারপাশে কী রয়েছে?  "
response = ask_question(follow_up_question)
print("Follow Up Response:", response)

Follow Up Response: জাতীয় প্রতীকের চারপাশে ধানের শীষ, তিনটি পাটপাতা এবং দুই পাশে দুটি তারা (তারকা) রয়েছে।


In [ ]:
# Example usage
follow_up_question = "এর  চারপাশে কয়টা তারকা রয়েছে? "
response = ask_question(follow_up_question)
print("Follow Up Response:", response)

Follow Up Response: দুই টা তারকা।


In [ ]:
# Example usage
question = "সংবিধানে মূলনীতি?  "
response = ask_question(question)
print("Response:", response)

Response: সংবিধানের মূলনীতি হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা।


In [ ]:
# Example usage
question = "রাষ্ট্রের  মূলনীতি?  "
response = ask_question(question)
print("Response:", response)

Response: উত্তর প্রাসঙ্গিক অংশ থেকে রাষ্ট্রের মূলনীতি হলো জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা। এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।


In [ ]:
# Example usage
question = "বাংলাদেশের মূলনীতি?  "
response = ask_question(question)
print("Response:", response)

Response: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা - এই নীতিসমূহ এবং এদের থেকে উদ্ভূত অন্যান্য নীতিসমূহ রাষ্ট্র পরিচালনার মূলনীতি হিসেবে গণ্য হবে।


In [ ]:
# Example usage
question = "বাংলাদেশের মূলনীতিসমূহ কী কী?  "
response = ask_question(question)
print("Response:", response)

Response: জাতীয়তাবাদ, সমাজতন্ত্র, গণতন্ত্র ও ধর্মনিরপেক্ষতা।


In [ ]:
# Example usage
question = "কোন ব্যাক্তি গ্রেফতার হলে তার অধিকারসমূহ কী কী ?  "
response = ask_question(question)
print("Response:", response)

Response: গ্রেপ্তার হলে ব্যক্তিকে নিকটতম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে, আইনজীবীর সঙ্গে পরামর্শ করা যাবে, আত্মপক্ষ সমর্থনের অধিকার থাকবে। আটক রাখা হলে অতিরিক্ত সময় আটক রাখা যাবে না।


In [ ]:
# Example usage
question = "তাকে কত সময়য়ের মধ্যে ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে?  "
response = ask_question(question)
print("Response:", response)

Response: গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে।


In [ ]:
# Example usage
question = "তাকে কতক্ষণ আটক রাখা যাবে? "
response = ask_question(question)
print("Response:", response)

Response: গ্রেপ্তার ব্যক্তিকে আটক রাখা যাবে না অতিরিক্ত সময়।


In [ ]:
# Example usage
question = "আটক করার পর করণীয় কী?  "
response = ask_question(question)
print("Response:", response)

Response: আটক করার পর করণীয় হলো গ্রেপ্তারকৃত বা প্রহরায় আটক ব্যক্তিকে গ্রেপ্তারের ২৪ ঘণ্টার মধ্যে নিকটম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে। ম্যাজিস্ট্রেটের অনুমতি ছাড়া তাকে অতিরিক্ত সময় আটক রাখা যাবে না।


In [ ]:
# Example usage
question = "কেউ আটক হলে তার অধিকার কী কী ? "
response = ask_question(question)
print("Response:", response)

Response: গ্রেপ্তার হলে ব্যক্তিকে নিকটতম ম্যাজিস্ট্রেটের সামনে হাজির করতে হবে, আইনজীবীর সঙ্গে পরামর্শ করা যাবে, আত্মপক্ষ সমর্থনের অধিকার থাকবে। আটক রাখা হলে অতিরিক্ত সময় আটক রাখা যাবে না।


In [ ]:
# Example usage
question = "তাকে কি আটক হওয়ার কারণ জানান হবে? কতো সময় পরে?  "
response = ask_question(question)
print("Response:", response)

Response: গ্রেপ্তার ও আটক সম্পর্কে রক্ষাকবচ অনুসারে গ্রেপ্তার হলে ব্যক্তিকে যত দ্রুত সম্ভব গ্রেপ্তার হওয়ার কারণ জানানো হবে, এবং তাকে নির্দিষ্ট করা সময়ের বেশি প্রহরায় আটক রাখা যাবে না। গ্রেপ্তার ব্যক্তির আইনজীবীর সঙ্গে পরামর্শ করার এবং আত্মপক্ষ সমর্থনের অধিকার থেকে বঞ্চিত করা যাবে না।


In [95]:
import pandas as pd

# Data
data = {
    "Question": [

        "দেশের সর্বোচ্চ আইন কী?",
        "কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?",
        "বাংলাদেশের জাতীয় সঙ্গীত কী?",
        "সংবিধানে রাষ্ট্রের মূলনীতি কী বলা হয়েছে?",
        "সংবিধানের সংশোধন কিভাবে হয়?"
    ],
    "Context": [

        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "যদি সংবিধান লঙ্ঘন হয়, তবে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।",
        "বাংলাদেশের জাতীয় সঙ্গীত 'আমার সোনার বাংলা'।",
        "রাষ্ট্রের মূলনীতি হিসেবে গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, এবং ধর্মনিরপেক্ষতা বলা হয়েছে।",
        "সংবিধানের সংশোধন করতে সংসদে বিশেষ প্রস্তাব পাস করতে হয়।"
    ],
    "Answer": [

        "বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।",
        "সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।",
        "'আমার সোনার বাংলা' জাতীয় সঙ্গীত।",
        "গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্মনিরপেক্ষতা।",
        "সংসদে বিশেষ প্রস্তাব পাসের মাধ্যমে সংশোধন হয়।"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV file
df.to_csv('bangla_constitution_part1_question_ans.csv', index=False)

print("CSV file with questions, context, and answers has been created successfully.")


CSV file with questions, context, and answers has been created successfully.


In [96]:
# prompt: load the csv file

import pandas as pd

# Assuming the CSV file 'bangla_constitution_qa.csv' is in the current directory
try:
    df = pd.read_csv('/content/bangla_constitution_part1_question_ans.csv')
    print("CSV file loaded successfully.")
    # Now you can work with the DataFrame 'df'
    #print(df.head())  # Print the first few rows to verify

except FileNotFoundError:
    print("Error: 'bangla_constitution_part1_question_ans.csv.csv' not found in the current directory.")
except pd.errors.EmptyDataError:
    print("Error: 'bangla_constitution_part1_question_ans.csv' is empty.")
except pd.errors.ParserError:
    print("Error: Unable to parse the CSV file. Check its format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


CSV file loaded successfully.


In [110]:
from sklearn.metrics.pairwise import cosine_similarity

#modell = SentenceTransformer('shihab17/bangla-sentence-transformer')
def evaluate_retrieval_accuracy(dataset, model, k=5, similarity_threshold=0.7):
    retrieval_hits = 0
    total_questions = len(dataset)

    for _, example in dataset.iterrows():  # Iterate over rows of the DataFrame
        question = example['Question']
        context = example['Context']
        true_answer = example['Answer']

        generated_ans=main_chain.invoke(question);

        print(question)
        print('question: ', question)
        print("answer ",true_answer)
        print("gen_ans ",generated_ans)


        print("--------------------------------------------------------------------------------------\n\n")


# Example usage with your DataFrame (df):
retrieval_accuracy = evaluate_retrieval_accuracy(df, model)
#print(f"Retrieval Accuracy: {retrieval_accuracy:.2f}")


দেশের সর্বোচ্চ আইন কী?
question:  দেশের সর্বোচ্চ আইন কী?
answer  বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans  সংবিধান হলো দেশের সর্বোচ্চ আইন।
--------------------------------------------------------------------------------------


কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
question:  কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
answer  সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
gen_ans  সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে যদি জরুরী অবস্থা ঘোষণা কার্যকর থাকে এবং সংবিধানের কোনো অনুচ্ছেদের বিধান স্থগিতকরণের প্রয়োজন হয়।
--------------------------------------------------------------------------------------


বাংলাদেশের জাতীয় সঙ্গীত কী?
question:  বাংলাদেশের জাতীয় সঙ্গীত কী?
answer  'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
gen_ans  আমার সোনার বাংলা।
--------------------------------------------------------------------------------------


সংবিধানে রাষ্ট্রের মূলনীতি কী বলা হয়েছে?
question:  সংবিধানে রাষ্ট্রের মূলনীতি কী বলা হয়েছে?
answer  গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্থা, ধর্ম

In [108]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os

class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        self.model = SentenceTransformer(model_name)

    def encode(self, sentences):
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))

    def similarity(self, embedding1, embedding2):
        return util.pytorch_cos_sim(embedding1, embedding2)

def get_gpt_similarity(question, true_answer, generated_answer, llm):
    # Prompt for GPT to score semantic similarity
    prompt = f"""
    Given the question, true answer, and generated answer, score how semantically similar the generated answer is to the true answer on a scale of 0 to 1, where 1 is identical in meaning and 0 is completely different.

    Question: {question}
    True Answer: {true_answer}
    Generated Answer: {generated_answer}

    Provide a score (0–1) and a brief explanation in the format:
    Score: <score>
    Explanation: <explanation>
    """

    # Use ChatOpenAI's invoke method
    messages = [
        SystemMessage(content="You are an evaluator of semantic similarity."),
        HumanMessage(content=prompt)
    ]
    try:
        response = llm.invoke(messages)
        response_text = response.content

        # Parse score from response
        score_line = [line for line in response_text.split('\n') if line.startswith("Score: ")]
        if score_line:
            score = float(score_line[0].replace("Score: ", ""))
            return score
        else:
            print("Warning: Could not parse GPT score. Returning 0.")
            return 0.0
    except Exception as e:
        print(f"Error calling GPT: {e}")
        return 0.0

def evaluate_retrieval_accuracy(dataset, model, llm, k=1, similarity_threshold=0.7):
    y_true = []
    y_pred = []
    recall_scores, precision_scores, f1_scores, llm_reteval_cos_scores, llm_reteval_gpt_scores = [], [], [], [], []
    total_questions = len(dataset)

    for _, example in dataset.iterrows():
        question = example['Question']
        true_answer = example['Answer']

        # Placeholder for main_chain (replace with main_chain.invoke(question))
        generated_ans = f"Generated answer for: {question}"

        y_true.append(true_answer)
        y_pred.append(generated_ans)

        # Compute cosine similarity
        true_answer_embedding = model.encode([true_answer])[true_answer]
        generated_answer_embedding = model.encode([generated_ans])[generated_ans]
        answer_similarity = cosine_similarity([true_answer_embedding], [generated_answer_embedding])[0][0]

        # Determine relevance (cosine similarity >= threshold)
        is_relevant = 1 if answer_similarity >= similarity_threshold else 0
        total_relevant = 1  # One true answer per question

        # Recall@1
        recall = is_relevant / total_relevant
        recall_scores.append(recall)

        # Precision@1
        precision = is_relevant / k
        precision_scores.append(precision)

        # F1@1
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

        # LLM-retEval (Cosine)
        llm_reteval_cos_scores.append(answer_similarity)

        # LLM-retEval (GPT)
        llm_reteval_gpt = get_gpt_similarity(question, true_answer, generated_ans, llm)
        llm_reteval_gpt_scores.append(llm_reteval_gpt)

        # Print results
        print(f"question: {question}")
        print(f"answer: {true_answer}")
        print(f"gen_ans: {generated_ans}")
        print(f"Answer Cosine Similarity: {answer_similarity:.3f}")
        print(f"Relevant (Cosine >= {similarity_threshold}): {is_relevant}")
        print(f"Recall@{k}: {recall:.3f}")
        print(f"Precision@{k}: {precision:.3f}")
        print(f"F1@{k}: {f1:.3f}")
        print(f"LLM-retEval (Cosine): {answer_similarity:.3f}")
        print(f"LLM-retEval (GPT): {llm_reteval_gpt:.3f}")
        print("--------------------------------------------------------------------------------------\n")

    # Aggregate results
    avg_recall = np.mean(recall_scores)
    avg_precision = np.mean(precision_scores)
    avg_f1 = np.mean(f1_scores)
    avg_llm_reteval_cos = np.mean(llm_reteval_cos_scores)
    avg_llm_reteval_gpt = np.mean(llm_reteval_gpt_scores)

    return {
        f"Recall@{k}": avg_recall,
        f"Precision@{k}": avg_precision,
        f"F1@{k}": avg_f1,
        "LLM-retEval (Cosine)": avg_llm_reteval_cos,
        "LLM-retEval (GPT)": avg_llm_reteval_gpt,
        "y_true": y_true,
        "y_pred": y_pred
    }


# Initialize models
model = BanglaSentenceTransformerSmall()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", api_key=os.getenv("OPENAI_API_KEY"))

# Calculate metrics
k = 1
results = evaluate_retrieval_accuracy(df, model, llm, k=k, similarity_threshold=0.7)

# Print aggregated results
print("\nAggregated Results:")
for metric, value in results.items():
    if metric not in ["y_true", "y_pred"]:
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

question: দেশের সর্বোচ্চ আইন কী?
answer: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans: Generated answer for: দেশের সর্বোচ্চ আইন কী?
Answer Cosine Similarity: 0.873
Relevant (Cosine >= 0.7): 1
Recall@1: 1.000
Precision@1: 1.000
F1@1: 1.000
LLM-retEval (Cosine): 0.873
LLM-retEval (GPT): 0.800
--------------------------------------------------------------------------------------

question: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
answer: সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
gen_ans: Generated answer for: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
Answer Cosine Similarity: 0.897
Relevant (Cosine >= 0.7): 1
Recall@1: 1.000
Precision@1: 1.000
F1@1: 1.000
LLM-retEval (Cosine): 0.897
LLM-retEval (GPT): 0.850
--------------------------------------------------------------------------------------

question: বাংলাদেশের জাতীয় সঙ্গীত কী?
answer: 'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
gen_ans: Generated answer for: বাংলাদেশের জাতীয় সঙ্গীত কী?
Answer Cosine Similarity: 0.848
R

In [109]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import os

class BanglaSentenceTransformerSmall:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        self.model = SentenceTransformer(model_name)

    def encode(self, sentences):
        sentence_embeddings = self.model.encode(sentences)
        return dict(zip(sentences, sentence_embeddings))

    def similarity(self, embedding1, embedding2):
        return util.pytorch_cos_sim(embedding1, embedding2)

def get_gpt_similarity(question, true_answer, generated_answer, llm):
    # Prompt for GPT to score semantic similarity
    prompt = f"""
    Given the question, true answer, and generated answer, score how semantically similar the generated answer is to the true answer on a scale of 0 to 1, where 1 is identical in meaning and 0 is completely different.

    Question: {question}
    True Answer: {true_answer}
    Generated Answer: {generated_answer}

    Provide a score (0–1) and a brief explanation in the format:
    Score: <score>
    Explanation: <explanation>
    """

    # Use ChatOpenAI's invoke method
    messages = [
        SystemMessage(content="You are an evaluator of semantic similarity."),
        HumanMessage(content=prompt)
    ]
    try:
        response = llm.invoke(messages)
        response_text = response.content

        # Parse score from response
        score_line = [line for line in response_text.split('\n') if line.startswith("Score: ")]
        if score_line:
            score = float(score_line[0].replace("Score: ", ""))
            return score
        else:
            print("Warning: Could not parse GPT score. Returning 0.")
            return 0.0
    except Exception as e:
        print(f"Error calling GPT: {e}")
        return 0.0

def evaluate_retrieval_accuracy(dataset, model, llm, k=1, similarity_threshold=0.7):
    y_true = []
    y_pred = []
    recall_scores, precision_scores, f1_scores, llm_reteval_cos_scores, llm_reteval_gpt_scores = [], [], [], [], []
    total_questions = len(dataset)

    for _, example in dataset.iterrows():
        question = example['Question']
        true_answer = example['Answer']

        # Placeholder for main_chain (replace with main_chain.invoke(question))
        generated_ans = f"Generated answer for: {question}"

        y_true.append(true_answer)
        y_pred.append(generated_ans)

        # Compute cosine similarity
        true_answer_embedding = model.encode([true_answer])[true_answer]
        generated_answer_embedding = model.encode([generated_ans])[generated_ans]
        answer_similarity = cosine_similarity([true_answer_embedding], [generated_answer_embedding])[0][0]

        # Determine relevance (cosine similarity >= threshold)
        is_relevant = 1 if answer_similarity >= similarity_threshold else 0
        total_relevant = 1  # One true answer per question

        # Recall@1
        recall = is_relevant / total_relevant
        recall_scores.append(recall)

        # Precision@1
        precision = is_relevant / k
        precision_scores.append(precision)

        # F1@1
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

        # LLM-retEval (Cosine)
        llm_reteval_cos_scores.append(answer_similarity)

        # LLM-retEval (GPT)
        llm_reteval_gpt = get_gpt_similarity(question, true_answer, generated_ans, llm)
        llm_reteval_gpt_scores.append(llm_reteval_gpt)

        # Print results
        print(f"question: {question}")
        print(f"answer: {true_answer}")
        print(f"gen_ans: {generated_ans}")
        print(f"Answer Cosine Similarity: {answer_similarity:.3f}")
        print(f"Relevant (Cosine >= {similarity_threshold}): {is_relevant}")
        print(f"Recall@{k}: {recall:.3f}")
        print(f"Precision@{k}: {precision:.3f}")
        print(f"F1@{k}: {f1:.3f}")
        print(f"LLM-retEval (Cosine): {answer_similarity:.3f}")
        print(f"LLM-retEval (GPT): {llm_reteval_gpt:.3f}")
        print("--------------------------------------------------------------------------------------\n")

    # Aggregate results
    avg_recall = np.mean(recall_scores)
    avg_precision = np.mean(precision_scores)
    avg_f1 = np.mean(f1_scores)
    avg_llm_reteval_cos = np.mean(llm_reteval_cos_scores)
    avg_llm_reteval_gpt = np.mean(llm_reteval_gpt_scores)

    return {
        f"Recall@{k}": avg_recall,
        f"Precision@{k}": avg_precision,
        f"F1@{k}": avg_f1,
        "LLM-retEval (Cosine)": avg_llm_reteval_cos,
        "LLM-retEval (GPT)": avg_llm_reteval_gpt,
        "y_true": y_true,
        "y_pred": y_pred
    }


# Initialize models
model = BanglaSentenceTransformerSmall()
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", api_key=os.getenv("OPENAI_API_KEY"))

# Calculate metrics
k = 3
results = evaluate_retrieval_accuracy(df, model, llm, k=k, similarity_threshold=0.7)

# Print aggregated results
print("\nAggregated Results:")
for metric, value in results.items():
    if metric not in ["y_true", "y_pred"]:
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

question: দেশের সর্বোচ্চ আইন কী?
answer: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
gen_ans: Generated answer for: দেশের সর্বোচ্চ আইন কী?
Answer Cosine Similarity: 0.873
Relevant (Cosine >= 0.7): 1
Recall@3: 1.000
Precision@3: 0.333
F1@3: 0.500
LLM-retEval (Cosine): 0.873
LLM-retEval (GPT): 0.800
--------------------------------------------------------------------------------------

question: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
answer: সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
gen_ans: Generated answer for: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
Answer Cosine Similarity: 0.897
Relevant (Cosine >= 0.7): 1
Recall@3: 1.000
Precision@3: 0.333
F1@3: 0.500
LLM-retEval (Cosine): 0.897
LLM-retEval (GPT): 0.800
--------------------------------------------------------------------------------------

question: বাংলাদেশের জাতীয় সঙ্গীত কী?
answer: 'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
gen_ans: Generated answer for: বাংলাদেশের জাতীয় সঙ্গীত কী?
Answer Cosine Similarity: 0.848
R

In [ ]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00


In [111]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness

# Load your CSV dataset
df = pd.read_csv('bangla_constitution_part1_question_ans.csv')

questions = df['Question'].tolist()
ground_truth_answers = df['Answer'].tolist()
retrieved_contexts = df['Context'].apply(lambda x: [x]).tolist()

# Generate answers by calling your LLM for each question
generated_answers = []
for q in questions:
    # Replace with your actual LLM call, e.g. main_chain.invoke(q)
    answer = main_chain.invoke(q)  # Make sure main_chain.invoke returns a string answer
    generated_answers.append(answer)

# Prepare dataset for evaluation
data_samples = {
    'question': questions,
    'answer': generated_answers,
    'contexts': retrieved_contexts,
    'ground_truth': ground_truth_answers
}
dataset = Dataset.from_dict(data_samples)

# Evaluate with RAGAS metrics
score = evaluate(dataset, metrics=[faithfulness, answer_correctness])

# Convert scores to pandas DataFrame for per-question scores
df_scores = score.to_pandas()

# Print scores question-wise
for i, question in enumerate(questions):
    correctness = df_scores.loc[i, 'answer_correctness']
    faithful = df_scores.loc[i, 'faithfulness']
    print(f"Question {i+1}: {question}")
    print(f"Answer {i+1}: {ground_truth_answers[i]}")
    print(f"Generated Answer: {generated_answers[i]}")
    print(f"Answer Correctness Score: {correctness:.3f}")
    print(f"Faithfulness Score: {faithful:.3f}")
    print('-' * 50)


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Question 1: দেশের সর্বোচ্চ আইন কী?
Answer 1: বাংলাদেশের সংবিধান দেশের সর্বোচ্চ আইন।
Generated Answer: সংবিধান দেশের সর্বোচ্চ আইন।
Answer Correctness Score: 0.745
Faithfulness Score: 1.000
--------------------------------------------------
Question 2: কি কারণে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে?
Answer 2: সংবিধান লঙ্ঘন হলে সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে।
Generated Answer: সাংবিধানিক পদক্ষেপ নেওয়া যেতে পারে যদি জরুরী অবস্থা ঘোষণা কার্যকর থাকে এবং সংবিধানের কোনো অনুচ্ছেদের বিধান স্থগিতকরণের প্রয়োজন হয়।
Answer Correctness Score: 0.231
Faithfulness Score: 0.000
--------------------------------------------------
Question 3: বাংলাদেশের জাতীয় সঙ্গীত কী?
Answer 3: 'আমার সোনার বাংলা' জাতীয় সঙ্গীত।
Generated Answer: বাংলাদেশের জাতীয় সঙ্গীত হলো "আমার সোনার বাংলা" গানটির প্রথম ১০টি লাইন।
Answer Correctness Score: 0.731
Faithfulness Score: 0.500
--------------------------------------------------
Question 4: সংবিধানে রাষ্ট্রের মূলনীতি কী বলা হয়েছে?
Answer 4: গণতন্ত্র, সমাজবাদী রাষ্ট্রব্যবস্